In [41]:
import requests
import os
import sys
import pandas as pd
import numpy as np

# import key sklearn ml algos
# import key sklearn metrics
# import 

from typing import Optional
import glob
# Add the parent directory of this notebook to sys.path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)


import datetime
import json
from tqdm import tqdm
import gc
# import ds_utils
import random
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


In [86]:
horse_result_file  = '../data/race-result-horse.csv'
race_result_file = '../data/race-result-race.csv'

horse_df = pd.read_csv(horse_result_file)
race_df = pd.read_csv(race_result_file)

race_df['race_course_track'] = race_df['race_course'] + '_' + race_df['track']
horse_df['finishing_position'] = horse_df['finishing_position'].fillna('NA')

In [92]:
clean_position = []
for i,row in horse_df.iterrows():
    row_finishing_position = row['finishing_position']
    if 'DH' in row_finishing_position:
        row_position = int(row_finishing_position[0])
    else:
        if str.isdigit(row_finishing_position):
            row_position = int(row_finishing_position)
        else:
            row_position = 99
    clean_position.append(row_position)
horse_df['clean_position'] = clean_position

In [93]:
horse_df['clean_position'].value_counts()

clean_position
1     2387
4     2375
3     2370
2     2368
6     2360
5     2359
7     2347
8     2341
9     2313
10    2260
11    2186
12    2020
13     995
14     838
99     670
Name: count, dtype: int64

In [90]:
str.isdigit('5')

True

In [ ]:
# feature note:
# race_course_track: the combination of race_course and track
# race course/track feature - sectional time statistics
# race cource - race course_track conditional ratio
# 
# horse feature -  ratio between actual weight, declared weight
# horse feature - length behind winner statistics
# horse feature - running position statistics
# horse feature - jockey statistics - avg finishing position (<99)
# horse feature - trainer statistic - avg finishing position (<99)
# horse feature - failed to finish ration - num <99 / num_races
 
# horse race feature - finish time/course length
# horse race feature - 



# ask perplexity for features 

# initial analysis

In [4]:
print(horse_df.shape)
print(race_df.shape)


(30189, 19)
(2367, 12)


In [5]:
print(horse_df.columns)
print(race_df.columns)


Index(['finishing_position', 'horse_number', 'horse_name', 'horse_id',
       'jockey', 'trainer', 'actual_weight', 'declared_horse_weight', 'draw',
       'length_behind_winner', 'running_position_1', 'running_position_2',
       'running_position_3', 'running_position_4', 'finish_time', 'win_odds',
       'running_position_5', 'running_position_6', 'race_id'],
      dtype='object')
Index(['src', 'race_date', 'race_course', 'race_number', 'race_id',
       'race_class', 'race_distance', 'track_condition', 'race_name', 'track',
       'sectional_time', 'incident_report'],
      dtype='object')


In [8]:
horse_df.sample(5)

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
28604,13,7.0,SOUTHERN LEGEND,A252,J Moreira,C Fownes,122,1121,1,7-1/2,6.0,5.0,13.0,NaN,1.10.68,2.7,NaN,NaN,2016-688
19859,13,3.0,MAJOR CONNECTION,V067,O Bosson,D J Hall,132,1079,5,9-3/4,8.0,14.0,13.0,NaN,1.11.08,40,NaN,NaN,2015-774
22911,12,1.0,CELESTIAL NOVA,V045,T H So,C S Shum,131,1142,5,12-3/4,7.0,8.0,12.0,NaN,1.11.86,99,NaN,NaN,2016-230
11364,13,10.0,GOLDEN OSMANTHUS,N245,C K Tong,T K Ng,120,1138,9,11,7.0,8.0,10.0,13.0,1.24.99,99,NaN,NaN,2015-112
26937,8,5.0,JE PENSE,V354,S Clipperton,C W Chang,126,1111,7,6,6.0,6.0,6.0,6.0,1.49.47,11,8.0,NaN,2016-546


In [ ]:
use_cols=['']
horse_df.sample(5)

In [81]:
horse_df['finishing_position'].describe()

count     30187
unique       36
top           1
freq       2361
Name: finishing_position, dtype: object

In [87]:
pd.isnull(horse_df['finishing_position']).sum()

0

In [88]:
horse_df['finishing_position'].value_counts()

finishing_position
1        2361
2        2354
3        2350
6        2346
5        2341
4        2340
7        2339
8        2329
9        2305
10       2260
11       2186
12       2020
13        995
14        838
WV        461
WV-A      102
4 DH       35
PU         29
UR         26
3 DH       20
5 DH       18
WX         16
2 DH       14
6 DH       14
8 DH       12
1 DH       12
FE         10
WX-A       10
10 DH       8
9 DH        8
7 DH        8
DNF         7
TNP         6
11 DH       4
12 DH       2
NA          2
DISQ        1
Name: count, dtype: int64

In [89]:
horse_df[horse_df['finishing_position'].str.contains('DH')]['finishing_position']

297      9 DH
298      9 DH
592      2 DH
593      2 DH
752      1 DH
         ... 
29328    9 DH
29665    5 DH
29666    5 DH
29945    4 DH
29946    4 DH
Name: finishing_position, Length: 155, dtype: object

In [79]:
horse_df['running_position_3'].describe()

count    29542.000000
mean         6.826992
std          3.724886
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_3, dtype: float64

In [80]:
horse_df['running_position_4'].describe()

count    16618.000000
mean         6.942472
std          3.798080
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: running_position_4, dtype: float64

In [77]:
horse_df['jockey'].value_counts()

jockey
J Moreira    1995
K Teetan     1761
Z Purton     1651
D Whyte      1608
N Callan     1590
             ... 
K Manning       1
T Ono           1
M Nunes         1
J Spencer       1
T Jarnet        1
Name: count, Length: 106, dtype: int64

In [78]:
horse_df['trainer'].value_counts()

trainer
A S Cruz          1751
C Fownes          1724
C H Yip           1692
J Moore           1654
Y S Tsui          1652
J Size            1499
C S Shum          1466
W Y So            1438
P F Yiu           1410
A T Millard       1349
K L Man           1337
T P Yung          1263
D E Ferraris      1251
P O'Sullivan      1195
A Lee             1182
C W Chang         1136
K W Lui           1132
R Gibson          1103
D J Hall          1090
D Cruz            1012
L Ho              1012
T K Ng             715
S Woods            509
A Schutz           496
H Fujiwara           8
N Hori               7
Y Ikee               5
D A Hayes            4
A Fabre              4
A P O'Brien          4
J Lau                3
M Saito              3
Sir M R Stoute       3
Barande-Barbe        3
P Bary               2
G Allendorf          2
K C Chong            2
Y C Fung             2
de Royer Dupre       2
G Enebish            2
M C Tam              2
M Sakaguchi          2
T Ozeki              2
K C

In [37]:
race_df['incident_report'].sample(1).values[0]

'\n                TRAVEL BROTHER was withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame left fore).  BORN IN CHINA was also withdrawn on 5.4.16 by order of the Stewards acting on veterinary advice (lame right fore).  Before being allowed to race again, TRAVEL BROTHER and BORN IN CHINA will be subjected to an official veterinary examination.\nFrom the outside barriers, MR GENUINE and DISCIPLES TWELVE were shifted across behind runners shortly after the start.\nApproaching the 800 Metres, MR GENUINE was retired from the race.  A veterinary inspection of MR GENUINE immediately following the race found that horse to be lame in its right hind leg.  Before being allowed to race again, MR GENUINE will be subjected to an official veterinary examination.\nPABLOSKY and PACKING LLAREGYB were sent for sampling.\n'

In [38]:
exclude_cols =['incident_report']
race_df.drop(exclude_cols, axis=1).sample(5)

,src,race_date,race_course,race_number,race_id,race_class,race_distance,track_condition,race_name,track,sectional_time
538,20150415-5.html,2015-04-15,Happy Valley,5,2014-539,Class 3,1200,GOOD TO FIRM,THE HONG KONG EXCHANGES CHALLENGE CUP (HANDICAP),"TURF - ""B"" COURSE",23.59 22.78 23.26
1548,20160706-8.html,2016-07-06,Happy Valley,8,2015-772,Class 2,1200,GOOD,LUGER HANDICAP,"TURF - ""A"" COURSE",23.34 22.64 23.11
714,20150621-7.html,2015-06-21,Sha Tin,7,2014-714,Class 3,1200,YIELDING,DERBY RESTAURANT & BAR HANDICAP,"TURF - ""C"" COURSE",24.28 22.95 22.81
1205,20160224-4.html,2016-02-24,Happy Valley,4,2015-429,Class 1,1650,GOOD,SHAN KWONG HANDICAP,"TURF - ""C"" COURSE",28.16 24.59 24.15 22.93
23,20140921-5.html,2014-09-21,Sha Tin,5,2014-023,Class 4,1600,GOOD TO FIRM,GUANGZHOU HANDICAP,"TURF - ""B+2"" COURSE",24.11 23.56 25.16 23.01


In [51]:
race_df['track_condition'].value_counts()

track_condition
GOOD                1296
GOOD TO FIRM         860
GOOD TO YIELDING     126
YIELDING              28
WET SLOW              22
FAST                  19
WET FAST              14
YIELDING TO SOFT       1
SOFT                   1
Name: count, dtype: int64

In [53]:
race_df['race_course'].value_counts()


race_course
Sha Tin         1513
Happy Valley     854
Name: count, dtype: int64

In [74]:
race_df['race_class'].value_counts()

race_class
Class 4                        859
Class 3                        756
Class 5                        339
Class 2                        223
Class 1                         51
Group One                       32
Hong Kong Group Three           25
Griffin Race                    16
Group Two                       14
Group Three                     12
Class 4 (Restricted)             9
Hong Kong Group One              9
Hong Kong Group Two              8
Restricted Race                  6
Class 4 (Special Condition)      6
Class 3 (Special Condition)      2
Name: count, dtype: int64

In [72]:
race_df['race_name'].sample(5)

408                                          KUT CHEONG HANDICAP
2362                                         MIRACULOUS HANDICAP
1967                                          BUTTERFLY HANDICAP
642     THE SPORTS CLUB DIAMOND JUBILEE CHALLENGE CUP (HANDICAP)
1177                                            HEATHER HANDICAP
Name: race_name, dtype: object

In [54]:
race_df['race_name'].value_counts()

race_name
LYNDHURST HANDICAP                                     6
SATURN HANDICAP                                        6
KAM TIN RIVER HANDICAP                                 5
WONG NAI CHUNG HANDICAP                                5
DANDELION HANDICAP                                     5
                                                      ..
TONG FUK HANDICAP                                      1
AUDEMARS PIGUET HANDICAP                               1
THE PRINCE JEWELLERY & WATCH PREMIER CUP (HANDICAP)    1
VACHERON CONSTANTIN HANDICAP                           1
MEDIC KINGDOM HANDICAP                                 1
Name: count, Length: 1084, dtype: int64

In [76]:
race_df['race_course_track'].value_counts()

race_course_track
Sha Tin_ALL WEATHER TRACK           287
Happy Valley_TURF - "A" COURSE      264
Sha Tin_TURF - "A" COURSE           256
Sha Tin_TURF - "C+3" COURSE         251
Sha Tin_TURF - "C" COURSE           242
Happy Valley_TURF - "B" COURSE      236
Sha Tin_TURF - "B+2" COURSE         207
Happy Valley_TURF - "C" COURSE      202
Sha Tin_TURF - "A+3" COURSE         200
Happy Valley_TURF - "C+3" COURSE    152
Sha Tin_TURF - "B" COURSE            70
Name: count, dtype: int64

In [50]:
# sentiment analysis on hourse 

rid = np.random.randint(1, len(race_df))
print(race_df['incident_report'][rid])


                AUDACITY was crowded for room on jumping between TEAM SWEET and FOREVER ACCURATE which shifted in.
Shortly after the start, GAINFULJET and REGENCY BABY bumped, resulting in both horses becoming badly unbalanced.
From the outside barrier, SUPER SWEET ORANGE was shifted across behind runners in the early stages.
After the 1200 Metres, SUPER SWEET ORANGE got its head up on a number of occasions when proving difficult to settle.
After the 800 Metres, SMART SALUTE was left racing wide and without cover.
Near the 500 Metres, WORLD RECORD got its head on the side and shifted out towards the heels of SUPER SWEET ORANGE.
Shortly after entering the Straight, DIVINE DIYA was steadied when disappointed for running between TEAM SWEET and EQUITY DOCTRINE.
The Stewards inquired into the reason for Apprentice K K Chiong being dislodged from SUPER MAN passing the 350 Metres.  After taking evidence from Apprentice Chiong, in the presence of her allocated trainer, Mr K W Lui, N Callan (G